In [1]:
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as elemTree
import re

In [2]:
def download(method, url, headers = None, param = None, data=None) :
    try :
        resp = requests.request(method, url, params=param, data=data, headers = headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e :
        print(e.response.status_code)
        print(e.response.reason)
    return resp

In [5]:
# 한 페이지에 있는 url들 가져오기
def getUrl(page):  
    url = 'http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=' + str(page) + '&mode=list'
    html = download('get', url)
    dom = BeautifulSoup(html.text, 'lxml')
    
    goboon = dom.select('tbody > tr')
    if page is 1:
        goboonList = goboon[3:]
    else:
        goboonList = goboon
        
    referUrl = list()
    for _ in goboonList:
        url = requests.compat.urljoin(url, _.findAll('td')[1].find('a')['href'])
        referUrl.append(url)
    
    return referUrl

In [7]:
# url 타고 들어가서 안에 있는 내용 가져오기
def getContents(url):
    html = download('get', url)
    dom = BeautifulSoup(html.text, 'lxml')

    # 제목에서 앞에 숫자 제외하고 뽑아내기
    t = dom.select_one('thead tr').text
    title = t.replace(t.split()[0], '').strip()
    
    itemList = dom.select('tbody tr td')[1].text.strip()
    typeList = dom.select('#contentsViewTitle2 tbody tr')[1:]

    dispute = dict()
    dispute['item'] = title
    dispute['itemList'] = itemList
    type_list = list()
    bigo_list = list()
    tmp2 = 0

    for _ in typeList:
        inside = dict()

        # 비고가 있는 td를 측정하기 위함(length가 3개면 비고있는 td)
        a = _.findAll("td")
        chk = len(a) # td의 개수 측정(유형 | 해결기준 | 비고)

        if chk is not 0:
            inside['type'] = a[0].text.strip().replace('\xa0', '').replace('\r\n', '')
            
            # 해당 유형의 해결기준이 위의 td의 해결기준일 경우
            if chk is 1:
                inside['refer'] = tmp
            
            elif chk > 1:
                # 해결기준이 있으면 refer에 추가(해결기준이 없는 text의 길이는 최대 3)
                if len(a[1].text) > 3:
                    inside['refer'] = a[1].text.strip().replace('\xa0', '').replace('\r\n', '')
                    tmp = a[1].text.strip().replace('\xa0', '').replace('\r\n', '') # 다음 유형(다음 td)을 위한 해결기준 저장
                
                # 위의 td의 비고와 같을 때
                if chk is 2:
                    # 저장된 비고가 존재한다면
                    if tmp2 is not 0:
                        inside['bigo'] = tmp2
                    
                # 비고가 있는 td면 bigo에 추가    
                elif chk is 3:
                    bigo = a[2].text.strip().replace('\xa0', '').replace('\r\n', '').replace('\n', '')
                    if len(bigo) is not 0:
                        inside['bigo'] = bigo
                        tmp2 = bigo # 다음 유형(다음 td)을 위한 비고 저장

            type_list.append(inside)

    dispute['rule'] = type_list
    return dispute


disputeRefer=[]
for page in range(1,16):
    
    urlList = getUrl(page)
    
    for url in urlList:
        print(url)
        res = getContents(url)
        disputeRefer.append(res)

print(disputeRefer)

http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847326
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847325
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847324
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847323
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847322
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847321
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847320
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=1&mode=view&boardSeq=847319
http://www.ccn.go.kr/boa

http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=7&mode=view&boardSeq=477
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=476
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=475
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=474
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=473
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=472
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=471
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=8&mode=view&boardSeq=470
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=

http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=14&mode=view&boardSeq=398
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=14&mode=view&boardSeq=397
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=396
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=395
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=394
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=393
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=1171398
http://www.ccn.go.kr/board2/board.ccn?gSiteCode=2&nMenuCode=63&gMenuCode=5&boardCode=372&page=15&mode=view&boardSeq=392
http://www.ccn.go.kr/board2/board.cc

In [8]:
len(disputeRefer)

150

In [9]:
disputeRefer[149]

{'item': '휴양콘도미니엄업',
 'itemList': '',
 'rule': [{'type': '1) 제공된 용역이 계약내용과 다른 경우', 'refer': '- 계약해제'},
  {'type': '2) 이용예정일 경과 후 공사 완공으로 인한 이용지연',
   'refer': '- 이용 예정일로부터 실 이용가능일까지의 지연일수에 해당하는 지연보상금 배상',
   'bigo': '*지연보상금 = (계약금+중도금)×지체이율×(이용지체일수÷365)'},
  {'type': '3) 부당한 이용료 징수', 'refer': '- 차액환급'}]}

In [10]:
#해결기준 전체 저장
import json


In [11]:
with open('solution_gijun.json', 'w') as json_file:
    json.dump(disputeRefer,json_file)

In [12]:
with open('solution_gijun.json') as json_file :
    data = json.load(json_file)

In [13]:
data[149]

{'item': '휴양콘도미니엄업',
 'itemList': '',
 'rule': [{'type': '1) 제공된 용역이 계약내용과 다른 경우', 'refer': '- 계약해제'},
  {'type': '2) 이용예정일 경과 후 공사 완공으로 인한 이용지연',
   'refer': '- 이용 예정일로부터 실 이용가능일까지의 지연일수에 해당하는 지연보상금 배상',
   'bigo': '*지연보상금 = (계약금+중도금)×지체이율×(이용지체일수÷365)'},
  {'type': '3) 부당한 이용료 징수', 'refer': '- 차액환급'}]}

------

In [14]:
#업종(category)만 뽑기

In [16]:
disputeRefer[149]['item']

'휴양콘도미니엄업'

In [19]:
caList=[]
for a in range(150) :
    caList.append(disputeRefer[a]['item'])
caList    

['가전제품설치업',
 '결혼중개업',
 '결혼준비대행업',
 '국제결혼중개',
 '경비용역업',
 '고시원운영업',
 '골프장',
 '공공서비스 (전기서비스)',
 '공공서비스 (전화서비스)',
 '공공서비스 (가스서비스)',
 '공산품 (가전제품)',
 '공산품 (사무용기기)',
 '공산품 (전기통신기자재)',
 '공산품 (시계)',
 '공산품 (재봉기)',
 '공산품 (광학제품)',
 '공산품 (아동용품)',
 '공산품 (텔레비전,TV)',
 '공산품 (전구)',
 '공산품 (가구)',
 '공산품 (스마트폰) - 명칭 불문하고 이동통신 3세대 이후의 모든 휴대전화 포함',
 '공산품 (전자담배)',
 '공산품 (자동차)',
 '공산품 (모터사이클)',
 '공산품 (자전거)',
 '공산품 (보일러)',
 '공산품 (농업용기계)',
 '공산품 (어업용기계)',
 '공산품 (농업용자재)',
 '공산품 (어구)',
 '공산품 (축산자재)',
 '공산품 (건축자재)',
 '공산품 (주방용품)',
 '공산품 (문구)',
 '공산품 (의복류)',
 '공산품 (우산류)',
 '공산품 (신발)',
 '공산품 (가죽제품)',
 '공산품 (악기)',
 '공산품 (타이어)',
 '공산품 (연탄)',
 '공산품 (가방류)',
 '공산품 (생활위생용품)',
 '공산품 (가발)',
 '공연업 (영화 및 비디오물 상영업 제외)',
 '공연업 (영화관람)',
 '농·수·축산물 (란류)',
 '농·수·축산물 (육류)',
 '농·수·축산물 (곡류)',
 '농·수·축산물 (과일)',
 '농·수·축산물 (야채류)',
 '농·수·축산물 (수산물류)',
 '농·수·축산물 (종묘등)',
 '동물사료',
 '대리운전',
 '모바일콘텐츠업',
 '문화용품 등 (귀금속·보석)',
 '문화용품 등 (악세사리)',
 '문화용품 등 (도서·음반)',
 '문화용품 등 (스포츠용품·레저용품)',
 '물품대여서비스업(렌탈서비스업)',
 '미용업 (피부미용업·모발미용업·네일서비스업·왁싱업)',
 '봉안시설',
 '

In [24]:
with open('category.json', 'w') as json_file:
    json.dump(caList,json_file)

In [25]:
with open('category.json') as json_file :
    cate = json.load(json_file)

In [27]:
cate[149]

'휴양콘도미니엄업'

------

In [28]:
#해당품목(item)

In [30]:
disputeRefer[149]['itemList']

''

In [36]:
itList=[]
for a in range(150):
    itList.append(disputeRefer[a]['itemList'].split(","))
itList

[[''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['VTR',
  ' 냉장고',
  ' 세탁기',
  ' 선풍기',
  '에어콘',
  ' 라디오',
  ' 녹음기',
  ' 전축',
  '전자렌지',
  ' 전기보온밥통 및 밥솥',
  '전기다리미',
  ' 전기주전자',
  ' 전기장판',
  '전기담요',
  ' 전기청소기',
  ' 전기난로',
  '전기프라이팬',
  ' 가습기',
  ' 헤드폰',
  '전기면도기',
  ' 식기세척기',
  ' 식기건조기',
  '헤어드라이어',
  ' 전기오븐',
  ' 전기약탕기',
  '전기냄비',
  ' 전기토스터',
  ' 환풍기',
  '전기머리인두',
  ' 전기믹서',
  '연탄가스배출기',
  ' 전기펌프',
  ' 쥬서기',
  '소형전압조정기',
  ' 전기탈수기',
  '형광등기구',
  ' 전기스탠드',
  ' 전기문걸이',
  '도어폰',
  ' 전기찜통',
  ' 전기온수보온기',
  '전기곤로',
  ' 전기조리기',
  ' 전기온수기',
  '온장고',
  ' 송풍기',
  ' 공기청정기',
  '누전감지기',
  ' 살수기',
  ' 냉수기',
  ' 제빙기',
  '방범경보기',
  ' 빙수기',
  ' 차임벨',
  '전자오락기',
  ' 석유난로',
  ' 안테나',
  '정수기',
  ' 온수기',
  ' 온수세정기(비데기)',
  '연수기',
  ' DVD플레이어',
  'MP3플레이어',
  ' Voice-pen',
  '핸드블랜더',
  ' 전기튀김기',
  ' 전기찜기',
  '할로플레이트 등'],
 ['복사기',
  ' 타자기',
  ' 팩시밀리',
  '금전등록기',
  ' 퍼스널컴퓨터 및컴퓨터주변기기',
  ' 워드프로세서',
  '계산기',
  ' 캐비넷',
  ' 파일링캐비넷',
  '제본기',
  ' 등사기',
  ' 컴퓨터소모품(룸팩',
  '디스켓)',
  ' 빔 프로젝트',
 

In [39]:
with open('item.json', 'w') as json_file:
    json.dump(itList,json_file)

In [40]:
with open('item.json') as json_file :
    item = json.load(json_file)

In [41]:
item[149]

['']